### Deep Ensamble MDN Toy Example ###

"We use the same network architecture: 1-hidden layer neural network with ReLU nonlinearity (Nair and Hinton, 2010), containing 50 hidden units for smaller datasets and 100 hidden units for the larger protein and Year Prediction MSD datasets"

"We trained for 40 epochs; we refer to (Herna ́ndez-Lobato and Adams, 2015) for further details about the datasets and the experimental protocol. We used 5 networks in our ensemble. Our results are shown in Table 1, along with the PBP and MC-dropout results reported in their respective papers"

"On some datasets, we observe that our method is slightly worse in terms of RMSE. We believe that this might be caused due to the het- eroscedastic regression training criterion, which optimises for NLL instead of MSE as discussed in the toy example in Figure 3."

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [ ]:
%matplotlib inline

/Users/ermanoarruda/virtualenvs/robotics/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
import os
import copy
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from aml_dl.mdn.model.tf_ensemble_mdn_model import EnsembleMDN

from aml_dl.gp.simple_gp import GaussianProcess, kernel

from IPython import display
import time

## Configs

In [ ]:
EXP_NAME = 'exp_ensemble'


adam_params = {
    'type': 'adam',
    'params': {'learning_rate' : 0.001, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08, 'use_locking': False}
}

network_params = {
    'n_ensembles': 5,
    'dim_input': 1, 
    'dim_output': 1,
    'n_hidden': [100],#[20,20,10],
    'k_mixtures': 1,
    'write_summary': False,
    'load_saved_model': False,
#     'model_dir': check_point_dir + '/inv/',
#     'model_name':'ensemble_model_inv_10_kernels.ckpt',
    'optimiser': adam_params,
#     'summary_dir':summary_dir+'/inv/',
    'device': '/cpu:0',
    'adv_epsilon': 0.01
}

In [ ]:
tf.set_random_seed(42)
np.random.seed(seed=42)¶
sess = tf.InteractiveSession()

In [ ]:
ensamble_mdn = EnsembleMDN(network_params,sess)

In [ ]:
noise_level = 0.1#0.00000001;
width = 1.0 # 0.5
scale = 30 # 30


gp = GaussianProcess(noise_level, width, scale, kernel)

In [ ]:
ensamble_mdn._init_model()

## Prepare data

In [ ]:
def fun_h(t, noise_gain=0.):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
    return np.cos(2*t - 1/2)/2 + np.cos(t) + 1 + noise

def fun_poly(t, noise_gain=0.):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
    return -0.01*(t ** 3) + np.square(t)*0.01 + noise

def fun_cubic_poly(t, noise_gain=0.):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
    return 0.01*(t ** 3) + 0.01*noise

# *
def fun_s(t, noise_gain=0.):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
    return (t-8) ** 3 - 5 * (t - 8)

def fun_c(t, noise_gain=0.0):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
        
    return t * np.cos(t) + noise

def fun_g(t, noise_gain=0.):
    if len(t.shape) > 1:
        noise = noise_gain*np.random.randn(t.shape[0],t.shape[1])
    else:
        noise = noise_gain*np.random.randn(t.shape[0])
    noise = noise_gain*np.random.randn(d1, d2)
    return np.divide(np.power(np.multiply((np.sin(t) - np.sin(2*t)/2 + np.sin(3*t)/3. - 
                                           np.sin(4*t)/4. + 4),t),2),(t+1)) + noise

def sawtoothxy(x,y):
    t, r = cart2pol(x,y) # change to polar coordinates
    h = fun_h(t)
    g = fun_g(r)
    f = np.multiply(g,h)
    return f

func_y = fun_h

In [ ]:
num_data_points = 2000 # 1000
x_values = np.linspace(-6, 6, num_data_points)
y_values = fun_cubic_poly(x_values, 3.0)

train_idx = np.random.choice(np.arange(300,1650),20)

plt.plot(x_values, y_values, 'dodgerblue', linewidth=3.0)
plt.plot(x_values[train_idx], y_values[train_idx], 'ro')
# print x_values[300:1450]
# print y_values[300:1450]

In [ ]:
def test_and_plot(i, loss):
    
    x_test = x_values#np.linspace(0, 12, num_data_points)
    y_test = y_values #fun_h(x_test, 0.0)
    
    out_mus = ensamble_mdn.run_op(sess, 'mus', np.expand_dims(x_test,axis=1))
    out_sigma = ensamble_mdn.run_op(sess, 'sigmas', np.expand_dims(x_test,axis=1))
    out_pis = ensamble_mdn.run_op(sess, 'pis', np.expand_dims(x_test,axis=1))
    
    mean_out, var_out = ensamble_mdn.forward(sess, np.expand_dims(x_test,axis=1))
    
    stddev = 3.0
    fig2 = plt.figure(1,figsize=(15,15))
    fig2.clf()
    ax = fig2.add_subplot(411)

    ypl = [float(mu-stddev*np.sqrt(sig)) for mu,sig in zip(mean_out,var_out)]
    yph = [float(mu+stddev*np.sqrt(sig)) for mu,sig in zip(mean_out,var_out)]

    ax.plot(x_test, y_test, color='r')

    ax.fill_between(x_test.tolist(), ypl, yph, facecolor='grey', interpolate=True, alpha=0.9)
    ax.plot(x_test, mean_out, color='b', linewidth=2.5)

#     colours = np.random.rand((network_params['n_ensembles'],3))#['g','c','k','m','y','']

    for k in range(network_params['n_ensembles']):
        ax.plot(x_test, np.reshape(out_mus[k],(-1,)), color=(np.random.rand(), np.random.rand(), np.random.rand()),linewidth=0.5)

    ax.scatter(x_train, y_train)

#     plt.show()
    
    plt.title('Iteration %d'%(i,))
    
    ax2 = fig2.add_subplot(412)
    mloss = np.mean(loss,axis=0)
    ax2.plot(mloss[:None], 'r-')
    
    ax3 = fig2.add_subplot(413)
    ax3.plot(x_test, np.sqrt(var_out), color='g')
    
    
    return fig2, mloss

In [ ]:
# x_train = x_values[350:650,None]
# y_train = y_values[350:650,None]
# x_train = x_values[600:900,None]
# y_train = y_values[600:900,None]

# x_train = x_values[753:1054,None]
# y_train = y_values[753:1054,None]

In [ ]:
# x_train = np.vstack([x_values[15:20,None],x_values[45:50,None],x_values[75:80,None]])
# y_train = np.vstack([y_values[15:20,None],y_values[45:50,None],y_values[75:80,None]])

In [ ]:
# x_train = np.vstack([x_values[200:400,None],x_values[750:1000,None],x_values[1250:1450,None]])
# y_train = np.vstack([y_values[200:400,None],y_values[750:1000,None],y_values[1250:1450,None]])

In [ ]:
x_train = np.vstack([x_values[train_idx,None]])
y_train = np.vstack([y_values[train_idx,None]])

In [ ]:
for idx in range(x_train.shape[0]):
    gp.update(x_train[idx,:], y_train[idx,:])

In [ ]:
pred, cov, logp = gp.predict3(x_values)
std = np.sqrt(np.diagonal(cov))*3.0

In [ ]:
sample_posterior = gp.sample_multivariate(pred, cov, len(pred), 5)

In [ ]:
fig2 = plt.figure(2, figsize=(8,8))
plt.errorbar(x_values, pred, yerr=std, fmt='r-', ecolor='g', elinewidth=0.25, linewidth=3.5)
plt.plot(x_values, y_values, 'dodgerblue', linewidth=3.0)
# plt.plot(x_values, sample_posterior)
# plt.xlim([0,12])
# plt.ylim([-0,3.0])
plt.plot(gp.x, gp.y, 'ro')

In [ ]:
stddev = 1.9600 #3.0
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(211)
ypl = [float(mu-stddev*np.sqrt(sig)) for mu,sig in zip(pred,std)]
yph = [float(mu+stddev*np.sqrt(sig)) for mu,sig in zip(pred,std)]

ax.plot(x_values, y_values, color='r')

ax.fill_between(x_values.tolist(), ypl, yph, facecolor='grey', interpolate=True, alpha=0.9)

ax2 = fig.add_subplot(212)

ax2.plot(x_values, np.sqrt(std), color='g')

In [ ]:
plt.figure()
plt.scatter(x_train,y_train)
plt.plot(x_values,y_values)
plt.show()

In [ ]:
iterations = 40

In [ ]:
mean_loss = []
for i in range(40):
    loss = ensamble_mdn.train(x_train, y_train, sess, iterations)

#     fig, mloss = test_and_plot(i, loss)
#     mean_loss.append(np.mean(mloss))
#     ax2 = fig.add_subplot(414)
#     ax2.plot(mean_loss, 'r-')
    
#     display.display(plt.gcf())
#     display.clear_output(wait=True)
#     time.sleep(0.001)
    
#     ax = fig2.add_subplot(212)
#     ax.plot(np.arange(100, iterations,1), loss[0, 100:], 'r-')

## Test

In [ ]:
x_test = x_values#np.linspace(0, 12, num_data_points)
y_test = y_values #fun_h(x_test, 0.0)

In [ ]:
out_mus = ensamble_mdn.run_op(sess, 'mus', np.expand_dims(x_test,axis=1))
out_sigma = ensamble_mdn.run_op(sess, 'sigmas', np.expand_dims(x_test,axis=1))
out_pis = ensamble_mdn.run_op(sess, 'pis', np.expand_dims(x_test,axis=1))

In [ ]:
# out_mus = np.reshape(out_mus,(-1,1))
# out_sigma = np.reshape(out_sigma,(-1,1))
# out_pis = np.reshape(out_pis,(-1,1))

In [ ]:
# print np.reshape(out_mus[0],(-1,1))
# print out_sigma.shape
# print out_pis.shape

In [ ]:
mean_out, var_out = ensamble_mdn.forward(sess, np.expand_dims(x_test,axis=1))

In [ ]:
stddev = 3.0#1.9600 #3.0

fig2 = plt.figure(2, figsize=(8,8))
# plt.xlim([0,12])
plt.ylim([-3.0,3.0])
plt.xlim([-6.0,6.0])
ax = fig2.add_subplot(111)

ypl = [float(mu-stddev*np.sqrt(sig[0])) for mu,sig in zip(mean_out,var_out)]
yph = [float(mu+stddev*np.sqrt(sig[0])) for mu,sig in zip(mean_out,var_out)]

plt.plot(x_values, y_values, 'dodgerblue', linewidth=3.0)


ax.fill_between(x_test.tolist(), ypl, yph, facecolor='grey', interpolate=True, alpha=0.9)

ax.plot(x_test.flatten(), mean_out.flatten(), color='b', linewidth=2.5)

colours = ['g','c','k','m','y']

for k in range(network_params['n_ensembles']):
    y_em = np.reshape(out_mus[k],(-1,))
    ax.plot(x_test, y_em, color=(np.random.rand(),np.random.rand(),np.random.rand()),linewidth=0.5)

# plt.scatter(x_train,y_train)
plt.plot(x_train, y_train, 'ro')

plt.show()